# Complete End-to-End Machine Learning Pipeline

This notebook demonstrates a production-ready machine learning pipeline including:
- Data cleaning and preprocessing
- Exploratory Data Analysis (EDA)
- Feature engineering
- Model training and evaluation
- Model deployment preparation

## Project Structure
```
machine-learning-pipeline/
│── data/
│   ├── raw/
│   ├── processed/
│── notebooks/
│── src/
│   ├── data_cleaning.py
│   ├── feature_engineering.py
│   ├── model_training.py
│   ├── model_evaluation.py
│   └── utils.py
│── models/
│── deployment/
│── plots/
│── docs/
│── README.md
│── requirements.txt
```

## 1. Environment Setup and Library Imports

First, let's import all necessary libraries and set up our environment.

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys
from pathlib import Path

# Machine Learning libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           confusion_matrix, classification_report, roc_auc_score,
                           mean_squared_error, mean_absolute_error, r2_score)
import xgboost as xgb
import pickle
import json

# Configure settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

print("All libraries imported successfully!")
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

## 2. Project Structure Creation

Let's create the required directory structure for our ML pipeline.

In [ ]:
# Create directory structure
def create_project_structure():
    """Create the complete project directory structure."""
    directories = [
        '../data/raw',
        '../data/processed',
        '../src',
        '../models',
        '../deployment',
        '../plots',
        '../docs'
    ]
    
    for directory in directories:
        Path(directory).mkdir(parents=True, exist_ok=True)
        print(f"✓ Created/verified directory: {directory}")

# Create the project structure
create_project_structure()

# Add src to path for imports
src_path = os.path.abspath('../src')
if src_path not in sys.path:
    sys.path.append(src_path)

print("\n✓ Project structure created successfully!")
print("✓ Source directory added to Python path")

## 3. Data Loading and Initial Exploration

We'll create a sample dataset for demonstration, then load and explore it.

In [ ]:
# Create sample dataset for demonstration
def create_sample_dataset(filepath, n_samples=500):
    """Create a sample dataset for loan approval prediction."""
    np.random.seed(42)
    
    data = {
        'age': np.random.randint(18, 80, n_samples),
        'income': np.random.normal(50000, 20000, n_samples),
        'education_years': np.random.randint(8, 20, n_samples),
        'experience': np.random.randint(0, 40, n_samples),
        'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'], n_samples),
        'gender': np.random.choice(['Male', 'Female'], n_samples),
        'married': np.random.choice(['Yes', 'No'], n_samples),
        'credit_score': np.random.randint(300, 850, n_samples),
        'loan_amount': np.random.normal(200000, 100000, n_samples),
        'employment_type': np.random.choice(['Full-time', 'Part-time', 'Self-employed', 'Unemployed'], n_samples)
    }
    
    # Create target variable (loan approval) based on logical rules
    df = pd.DataFrame(data)
    df['loan_approved'] = (
        (df['income'] > 40000) & 
        (df['credit_score'] > 650) & 
        (df['employment_type'].isin(['Full-time', 'Self-employed']))
    ).astype(int)
    
    # Add some missing values randomly
    missing_cols = ['income', 'credit_score', 'education_years']
    for col in missing_cols:
        missing_idx = np.random.choice(df.index, size=int(0.05 * len(df)), replace=False)
        df.loc[missing_idx, col] = np.nan
    
    # Add some duplicates
    duplicate_rows = df.sample(n=int(0.02 * len(df)))
    df = pd.concat([df, duplicate_rows], ignore_index=True)
    
    # Save the dataset
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    df.to_csv(filepath, index=False)
    print(f"Sample dataset created with {len(df)} rows and saved to {filepath}")
    return df

# Create sample dataset
sample_path = "../data/raw/sample_data.csv"
sample_df = create_sample_dataset(sample_path)

print(f"\nDataset created successfully!")
print(f"Shape: {sample_df.shape}")
print(f"Columns: {list(sample_df.columns)}")

In [ ]:
# Load and explore the dataset
df = pd.read_csv("../data/raw/sample_data.csv")

print("=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n=== COLUMN INFORMATION ===")
print(df.info())

print("\n=== FIRST FEW ROWS ===")
display(df.head())

print("\n=== STATISTICAL SUMMARY ===")
display(df.describe(include='all'))

print("\n=== MISSING VALUES ===")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_summary = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_percent
})
print(missing_summary[missing_summary['Missing Count'] > 0])

print("\n=== DUPLICATE ROWS ===")
print(f"Number of duplicate rows: {df.duplicated().sum()}")

print("\n=== TARGET VARIABLE DISTRIBUTION ===")
if 'loan_approved' in df.columns:
    print(df['loan_approved'].value_counts())
    print(f"\nApproval rate: {df['loan_approved'].mean():.2%}")

## 4. Data Cleaning Pipeline

Now we'll implement comprehensive data cleaning using our modular approach.

In [ ]:
# Import our custom data cleaning module
from data_cleaning import DataCleaner

# Initialize data cleaner
cleaner = DataCleaner()

# Run complete data cleaning pipeline
input_path = "../data/raw/sample_data.csv"
output_path = "../data/processed/cleaned_data.csv"

cleaned_df = cleaner.clean_data(
    filepath=input_path,
    output_path=output_path,
    missing_strategy={'income': 'median', 'credit_score': 'median'},
    outlier_method='iqr',
    outlier_action='cap'
)

print(f"\n✓ Data cleaning complete!")
print(f"Original shape: {df.shape}")
print(f"Cleaned shape: {cleaned_df.shape}")
print(f"Cleaned data saved to: {output_path}")

## 5. Exploratory Data Analysis (EDA)

Let's perform comprehensive EDA to understand our cleaned data better.

In [ ]:
# Import EDA module
from eda import EDAAnalyzer

# Initialize EDA analyzer
eda_analyzer = EDAAnalyzer(plots_dir="../plots")

# Generate comprehensive EDA report
eda_report = eda_analyzer.generate_eda_report(cleaned_df, target_col='loan_approved')

print("✓ EDA analysis complete!")
print(f"Generated plots: {eda_report['plots_generated']}")
print("\nCheck the '../plots' directory for all visualizations!")

# Display summary statistics
print("\n=== SUMMARY STATISTICS ===")
summary_stats = eda_analyzer.generate_summary_statistics(cleaned_df)
print(f"Dataset shape: {summary_stats['dataset_info']['shape']}")
print(f"Missing values: {summary_stats['dataset_info']['missing_values']}")
print(f"Duplicate rows: {summary_stats['dataset_info']['duplicates']}")

## 6. Feature Engineering

Now we'll apply comprehensive feature engineering transformations.

In [ ]:
# Import feature engineering module
from feature_engineering import FeatureEngineer

# Initialize feature engineer
feature_engineer = FeatureEngineer(models_dir="../models")

# Run complete feature engineering pipeline
X_train, X_test, y_train, y_test = feature_engineer.engineer_features(
    cleaned_df, 
    target_col='loan_approved',
    scaling_method='standard',
    test_size=0.2,
    random_state=42
)

print(f"✓ Feature engineering complete!")
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Original features: {len(cleaned_df.columns)}")
print(f"Engineered features: {len(X_train.columns)}")
print(f"Features added: {len(X_train.columns) - len(cleaned_df.columns)}")

# Save processed data
X_train.to_csv("../data/processed/X_train.csv", index=False)
X_test.to_csv("../data/processed/X_test.csv", index=False)
y_train.to_csv("../data/processed/y_train.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)

print("✓ Processed data saved to ../data/processed/")

## 7. Model Training

We'll train multiple models and compare their performance.

In [ ]:
# Import model training module
from model_training import ModelTrainer

# Initialize model trainer
trainer = ModelTrainer(models_dir="../models")

# Train all models
training_results = trainer.train_all_models(
    X_train, y_train, 
    use_grid_search=True, 
    cv_folds=5
)

print(f"✓ Model training complete!")
print(f"Models trained: {len(training_results)}")

# Display training results summary
for model_name, result in training_results.items():
    if result['status'] == 'success':
        print(f"\n{model_name}:")
        print(f"  ✓ Best CV Score: {result['best_cv_score']:.4f}")
        print(f"  ✓ Best Parameters: {result['best_params']}")
    else:
        print(f"\n{model_name}:")
        print(f"  ✗ Training failed: {result.get('error', 'Unknown error')}")

# Quick model comparison on test set
comparison = trainer.compare_models_quick(X_test, y_test)
print(f"\n=== MODEL COMPARISON ON TEST SET ===")
display(comparison)

## 8. Model Evaluation and Comparison

Let's perform comprehensive model evaluation with detailed metrics and visualizations.

In [ ]:
# Import model evaluation module
from model_evaluation import ModelEvaluator

# Initialize model evaluator
evaluator = ModelEvaluator(models_dir="../models", plots_dir="../plots")

# Run comprehensive evaluation
evaluation_results = evaluator.evaluate_all_models(X_test, y_test)

print(f"✓ Model evaluation complete!")
print(f"Models evaluated: {len(evaluation_results)}")

# Display detailed results
for model_name, result in evaluation_results.items():
    print(f"\n=== {model_name.upper()} EVALUATION ===")
    metrics = result['metrics']
    
    if result['problem_type'] == 'classification':
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1-Score: {metrics['f1_score']:.4f}")
        if metrics['roc_auc']:
            print(f"ROC-AUC: {metrics['roc_auc']:.4f}")
    else:
        print(f"RMSE: {metrics['rmse']:.4f}")
        print(f"MAE: {metrics['mae']:.4f}")
        print(f"R² Score: {metrics['r2_score']:.4f}")

print(f"\n✓ Evaluation plots saved to: ../plots/")
print(f"✓ Detailed results saved to: ../models/evaluation_report.json")

## 9. Best Model Selection and Saving

Let's identify and save the best performing model for deployment.

In [ ]:
# Best model information is already saved by the evaluator
# Let's load and display the best model info

with open("../models/best_model_info.json", "r") as f:
    best_model_info = json.load(f)

print("=== BEST MODEL INFORMATION ===")
for problem_type, info in best_model_info.items():
    print(f"\n{problem_type.upper()}:")
    print(f"  Best Model: {info['model_name']}")
    for metric, value in info['all_metrics'].items():
        if value is not None:
            print(f"  {metric.title()}: {value:.4f}")

# Verify that best model is saved
import os
best_model_path = "../models/best_model.pkl"
if os.path.exists(best_model_path):
    print(f"\n✓ Best model saved at: {best_model_path}")
    
    # Load and test the best model
    with open(best_model_path, "rb") as f:
        best_model = pickle.load(f)
    
    # Make a sample prediction
    sample_prediction = best_model.predict(X_test.head(1))
    print(f"✓ Best model loaded and tested successfully")
    print(f"Sample prediction: {sample_prediction[0]}")
else:
    print("✗ Best model file not found")

## 10. Flask API Deployment Setup

Let's set up our Flask API for model deployment. The API files are already created in the deployment folder.

In [ ]:
# The Flask API is already created in ../deployment/app.py
# Let's verify the deployment files exist and show how to use them

deployment_files = [
    "../deployment/app.py",
    "../deployment/test_api.py"
]

print("=== DEPLOYMENT FILES STATUS ===")
for file_path in deployment_files:
    if os.path.exists(file_path):
        print(f"✓ {file_path} exists")
    else:
        print(f"✗ {file_path} missing")

print(f"\n=== API USAGE INSTRUCTIONS ===")
print(f"""
To start the Flask API:
1. Navigate to the deployment directory:
   cd ../deployment

2. Run the Flask application:
   python app.py

3. The API will be available at: http://localhost:5000

Available endpoints:
- GET  /health          - Health check
- GET  /model_info      - Model information  
- POST /predict         - Make predictions
- GET  /predict_sample  - Test with sample data

To test the API:
- In another terminal, run: python test_api.py
""")

# Create a sample request for demonstration
sample_request = {
    "data": {
        "age": 35,
        "income": 55000,
        "education_years": 16,
        "experience": 10,
        "city": "New York",
        "gender": "Female",
        "married": "Yes",
        "credit_score": 720,
        "loan_amount": 200000,
        "employment_type": "Full-time"
    }
}

print(f"\nExample API request body:")
print(json.dumps(sample_request, indent=2))

## 11. Testing the Complete Pipeline

Let's verify that our complete ML pipeline works end-to-end and generate the final project files.

In [ ]:
# Verify all pipeline components and files
import glob

print("=== PIPELINE VERIFICATION ===")

# Check data files
data_files = [
    "../data/raw/sample_data.csv",
    "../data/processed/cleaned_data.csv",
    "../data/processed/X_train.csv", 
    "../data/processed/X_test.csv",
    "../data/processed/y_train.csv",
    "../data/processed/y_test.csv"
]

print(f"\nData Files:")
for file_path in data_files:
    if os.path.exists(file_path):
        size = os.path.getsize(file_path)
        print(f"✓ {file_path} ({size:,} bytes)")
    else:
        print(f"✗ {file_path} missing")

# Check model files
model_files = glob.glob("../models/*.pkl") + glob.glob("../models/*.json")
print(f"\nModel Files ({len(model_files)} files):")
for file_path in sorted(model_files):
    size = os.path.getsize(file_path)
    print(f"✓ {os.path.basename(file_path)} ({size:,} bytes)")

# Check plot files
plot_files = glob.glob("../plots/*.png") + glob.glob("../plots/*.json")
print(f"\nPlot Files ({len(plot_files)} files):")
for file_path in sorted(plot_files):
    size = os.path.getsize(file_path)
    print(f"✓ {os.path.basename(file_path)} ({size:,} bytes)")

# Check source files
src_files = glob.glob("../src/*.py")
print(f"\nSource Files ({len(src_files)} files):")
for file_path in sorted(src_files):
    size = os.path.getsize(file_path)
    print(f"✓ {os.path.basename(file_path)} ({size:,} bytes)")

# Check deployment files  
deploy_files = glob.glob("../deployment/*.py")
print(f"\nDeployment Files ({len(deploy_files)} files):")
for file_path in sorted(deploy_files):
    size = os.path.getsize(file_path)
    print(f"✓ {os.path.basename(file_path)} ({size:,} bytes)")

print(f"\n✅ Pipeline verification complete!")
print(f"✅ All components are functioning correctly!")

In [ ]:
# Generate requirements.txt file
requirements = [
    "pandas>=1.3.0",
    "numpy>=1.21.0", 
    "scikit-learn>=1.0.0",
    "xgboost>=1.5.0",
    "matplotlib>=3.3.0",
    "seaborn>=0.11.0",
    "flask>=2.0.0",
    "requests>=2.25.0"
]

with open("../requirements.txt", "w") as f:
    for req in requirements:
        f.write(req + "\n")

print("✓ requirements.txt created")

# Generate README.md file
readme_content = """# Machine Learning Pipeline Project

A complete end-to-end machine learning pipeline with data cleaning, feature engineering, model training, evaluation, and deployment.

## Project Structure

```
machine-learning-pipeline/
│── data/
│   ├── raw/                    # Raw data files
│   ├── processed/              # Processed data files
│── notebooks/
│   └── ml_pipeline_complete.ipynb  # Complete pipeline notebook
│── src/
│   ├── data_cleaning.py        # Data cleaning functions
│   ├── feature_engineering.py  # Feature engineering functions
│   ├── model_training.py       # Model training functions
│   ├── model_evaluation.py     # Model evaluation functions
│   └── utils.py               # Utility functions
│── models/                     # Trained models and metadata
│── deployment/
│   ├── app.py                 # Flask API application
│   └── test_api.py            # API testing script
│── plots/                      # Generated visualizations
│── docs/                       # Documentation
│── requirements.txt            # Python dependencies
└── README.md                   # This file
```

## Features

- **Data Cleaning**: Handle missing values, duplicates, outliers
- **Exploratory Data Analysis**: Comprehensive visualizations and statistics
- **Feature Engineering**: One-hot encoding, scaling, feature creation
- **Model Training**: Multiple algorithms with hyperparameter tuning
- **Model Evaluation**: Comprehensive metrics and comparisons
- **Deployment**: Flask API for real-time predictions

## Quick Start

1. **Install dependencies:**
   ```bash
   pip install -r requirements.txt
   ```

2. **Run the complete pipeline:**
   ```bash
   jupyter notebook notebooks/ml_pipeline_complete.ipynb
   ```

3. **Or run individual modules:**
   ```bash
   cd src
   python data_cleaning.py
   python feature_engineering.py
   python model_training.py
   python model_evaluation.py
   ```

4. **Start the API server:**
   ```bash
   cd deployment
   python app.py
   ```

5. **Test the API:**
   ```bash
   cd deployment
   python test_api.py
   ```

## API Usage

The Flask API provides the following endpoints:

- `GET /health` - Health check
- `GET /model_info` - Model information
- `POST /predict` - Make predictions
- `GET /predict_sample` - Test with sample data

Example prediction request:
```json
{
  "data": {
    "age": 35,
    "income": 55000,
    "education_years": 16,
    "experience": 10,
    "city": "New York",
    "gender": "Female", 
    "married": "Yes",
    "credit_score": 720,
    "loan_amount": 200000,
    "employment_type": "Full-time"
  }
}
```

## Models Supported

- Logistic Regression (Classification)
- Random Forest (Classification/Regression)
- XGBoost (Classification/Regression)

## Data Requirements

The pipeline is designed to work with any CSV dataset containing:
- Mixed numerical and categorical features
- 300+ rows recommended
- Target variable for supervised learning

## Contributing

1. Fork the repository
2. Create a feature branch
3. Make your changes
4. Add tests
5. Submit a pull request

## License

This project is licensed under the MIT License.
"""

with open("../README.md", "w") as f:
    f.write(readme_content)

print("✓ README.md created")
print("\n🎉 Complete ML Pipeline Setup Finished!")
print("\n📁 Project files generated:")
print("   ✓ requirements.txt")
print("   ✓ README.md") 
print("   ✓ All source modules in /src")
print("   ✓ Flask API in /deployment")
print("   ✓ Trained models in /models")
print("   ✓ Visualizations in /plots")
print("\n🚀 Your ML pipeline is ready to use!")